In [15]:
import pandas as pd
import numpy as np

import category_encoders as ce  # new lib for encoding categorical variables

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

from summarytools import dfSummary

In [3]:
heart_df = pd.read_csv('heart.csv')
dfSummary(heart_df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Age[int64],Mean (sd) : 53.5 (9.4)min < med < max:28.0 < 54.0 < 77.0IQR (CV) : 13.0 (5.7),50 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAClUlEQVR4nO3dQY6iQBSA4VdDa1ATiDHRnReYpBcuPcQcdo4wC0/gDdyZGIKRaSIivZh04kx6uhGotl7xf3tsFn8bgaqHqapKANd9e/QJAHU8PfoEfGSMGYnIsOHh56qqXro8Hx8QaseMMaPZbPYjiqJpk+OPx2NijPlJrH8j1O4NoyiartfrlziO83sOTNM03Gw208PhMBQRQr1BqJbEcZzP5/PfDQ4ddX4yHuBiCioQKlQgVKhAqFCBUKECV/2OKctyICKRMabpR3j5wIBQHZJl2aAoiuflchkEQXDXPdg3vj4wIFSHnM/npzAMx6vVKl8sFsm9x/v8wIBQHTSZTJo+LBDx9IEBF1NQgVChAqFCBUKFCoQKFbjq/48Wq/Sj6/U66Pp8+o5Q39FmlX5RFGFZlt/zPP8lIk1vMeEfhPq+xqv0d7vddLvdji+XS2Dr5PqIUD/QZJV+kiRe3nB/NC6moAKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUK3j7rbzlMl6V6jvEy1LbDdDUv1Ws5wMLZ4RVehiotlumJ6F2q13aAhcvDK3wNVUSaD9PVulSvzQAL14dXeB1qX7UYYOHsP6jTobJvCW+cDZV9S7jlbKjCviXccDlUEWHfEv7gyRRUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoYLzi1LwdVx+YbDVUNkJqofrLwy2Fmqfd4Jq5PoLg21+o/ZyJ6h2rr4w2Ppv1L7tBIUdXPVDBUKFCoQKFbiPis7YnHv1aagMgUAdtudefRgqQyBQl+25V599ozIEAnexNfeq1m9UhkDg0WqFmqZpeO8Hn06nUEQky7Jwv9+Pv/J4rX9b63m3Pb5OX6/+sLLp9hMoWwAAAABJRU5ErkJggg=="">",0(0.0%)
2,Sex[object],1. M2. F,725 (79.0%)193 (21.0%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAAuCAYAAAA/ZmtKAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABNUlEQVR4nO3bwYnDMBCF4dFiwuYiYQwuxkVssVuEujHGOiQhF+WQGvRmwP9XwUD+BGL8Uu/dAIVkZnczu3kfguHevfen5wHTsix/OefZ8wiM11o7Ukr/nsFNOed527ZnKeXldQTGOs/zt9Y67/t+MzO/2MzMSimvdV0fXkdA4u59wI/3AbgOYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZCaz78t13odgnCif79RaO2qtswV4uQ7jtNYOM3t73sDg5TrcBy+JKR9UovyyuX/rMF6IKV+EmRnGc5/yRZmZYbwoUz7+CV8AD3UhQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhlig4z7lC/KzAzjhZjyRZiZYTwGL5D5AApbat9q39BWAAAAAElFTkSuQmCC"">",0(0.0%)
3,ChestPainType[object],1. ASY2. NAP3. ATA4. TA,496 (54.0%)203 (22.1%)173 (18.8%)46 (5.0%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABcCAYAAAB5jMeAAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB70lEQVR4nO3bQWrbUBiF0V/FhGYiYQxejBfRxXYR2o0x1iAJmSiDzlrqZCDuU6RzNuCH+ZCQ0O3meS5I+NH6AOyH2Ijpquq5qp5aH2RD3ud5fm19iDU6nE6nX33fH1sfZCumabp1XfdbcP869H1/vFwur8MwvLU+zHd3v99/juN4vF6vT1Ultr8cqqqGYXg7n88vrQ+zEc+tD7BWHhCIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjZhD1Z9PY1ofZAv8j48dpmm6jeN4LJ/GLGKapltVvbc+xxr5LHx5Pgv/j86UjxQPCMSs6Tbq9rNxq1lXWSVt3yrWVVZJ+7CmdZVXLxvnAYEYsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMauY8rX+fTJWM+Uzgds+GwRiTPmIaXVlcxXboSbrKkuqfYqvqyyp9qvVusqSaoe81CVGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixESM2YsRGTHxdZUm1X03WVZZU+2SDQIx1FTFfubK5CrGIT9dVllAs5eG6yhKKJX1lXWUJxSK81CVGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixESM2YsRGzMN1lSUUS/p0XWUJxVJsEIixriLGAwIxH7NY2LncjNTWAAAAAElFTkSuQmCC"">",0(0.0%)
4,RestingBP[int64],Mean (sd) : 132.4 (18.5)min < med < max:0.0 < 130.0 < 200.0IQR (CV) : 20.0 (7.2),67 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACU0lEQVR4nO3dP47aQBiG8W/CwtqAbCEU6LhAJApKDpHD5ggpOAHtVnQWCBnhrIMxThdppUSJ/wm/1vPrB0/xNAP4G1cUhQFd9+nZGwD+x8uzN4CPnHO+mY1qfMStKIr3pvbTFYTaIc45fz6ffw2CYFb1My6Xy9k5961vsRJqt4yCIJhtt9v3MAzTsovjOPZ2u93sdDqNzIxQ0a4wDNPFYvGj4nK/0c10BIcpSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSGD2VM/keT40s8A5V2V5Z0dWEmqPJEkyzLJsvVqtBoPBoPQ0wC6PrCTUHrndbi+e5403m026XC7PZdZ2fWQlofbQZDKpOraysyMrOUxBAqFCAqFCAqFCAqFCAqFCAqFCAqFCAqFCAr9MtaDGfabB4/EYNr2fPiDUhtW5zzTLMi/P8y9pmn43s6o39/USoTav8n2mh8Nhtt/vx/f7fdDW5lQRakuq3Gd6Pp87+6eQZ+MwBQmECgmECgmECgmECgmECgmECgmECgl84Y/fag6vMGtxgAWhwszqD68wa3eABaHCzOoNrzBrf4AFoeKDGsMrzFocYMFhChIIFRIIFRIIF

In [ ]:
X = heart_df.drop('HeartDisease', axis=1)
y = heart_df['HeartDisease']

X, y

Antes de fazer qualquer anteração com os dados (preprocessamento) com os dados para a construção do modelo, vamos antes separar eles em treino e validação.

Se separarmos depois de fazer o preprocessamento, preencher colunas vazias por exemplo, o nosso dado treino vai "ver" o dado de teste, e isso vai prejudicar o modelo.

In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y) 

In [28]:
cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
num_cols = [col for col in X_train.columns if X_train[col].dtype == 'int64']

cat_cols, num_cols

(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'],
 ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR'])

In [29]:
# usados para fazer transformações em tipos diferentes de colunas
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', ce.OneHotEncoder())
])

transformer = ColumnTransformer([
    ("cat_transform", cat_pipe, cat_cols),
    ("num_transform", SimpleImputer(strategy='median'), num_cols)
])

In [31]:
X_train_transform = transformer.fit_transform(X_train, y_train)
X_valid_transform = transformer.transform(X_valid) 

In [35]:
tree = DecisionTreeClassifier()

In [37]:
tree.fit(X_train_transform, y_train)
predict = tree.predict(X_valid_transform)

In [38]:
import sklearn.metrics as skmetrics

In [41]:
print(f"Acurária: {round(skmetrics.accuracy_score(y_valid, predict) * 100, 2)}%")

Acurária: 76.96%
